In [1]:
import pandas as pd # Tratamiento de datos
import numpy as np
import spacy
nlp = spacy.load('es_dep_news_trf')
from spacy.lang.es.stop_words import STOP_WORDS
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer #Preprocesado y modelado
from sklearn.naive_bayes import MultinomialNB #Preprocesado y modelado
from flask import Flask, jsonify, request
import pickle

In [2]:
def analizer(x):
    return x

In [3]:
fileObject = open('modelo_balanceado_final', 'rb')

# Unpickle the data
NB_model = pickle.load(fileObject)

# Close file
fileObject.close()

In [4]:
FileObject = open('vectorizer_balanceado_final', 'rb')

Vectoryzer_model = pickle.load(FileObject)

FileObject.close()

In [5]:
FileObject = open('X_balanceado_final', 'rb')

X_balanceado = pickle.load(FileObject)

FileObject.close()

In [6]:
FileObject = open('Y_balanceado_final', 'rb')

Y_balanceado = pickle.load(FileObject)

FileObject.close()

In [7]:
def remove_accents(raw_text):

    raw_text = re.sub(u"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    return raw_text

def cleanV2(sentence):
    New_sentence = ''
    table = str.maketrans({key: None for key in '``¡!"#$%&\¿?()*+,-./:;<=>~^@[\\]_{|}'})
    for i in sentence:
        New_sentence = New_sentence + remove_accents(i)
    sentence = New_sentence
    sentence = re.sub('[^a-zA-Z]', ' ',sentence)
    doc = nlp(sentence)
    sentence=list(filter(None, [w.lower().translate(table) for w in sentence]))
    sentence = [token.lemma_ for token in doc if (token.pos_=='NOUN' or token.pos_=='VERB' or token.pos_=="ADJ" or token.pos_=="PROPN")] #or token.pos_=="INTJ")]
    sentence = [w for w in sentence if w not in STOP_WORDS]
    return sentence

def Corpus_lista (X):
    Lista_Corpus = X.tolist() 
    clean_corpus = []
    CORPUS = []
    for i in range(len(Lista_Corpus)):
        C = cleanV2(Lista_Corpus[i])
        if len(C)>=1:
            clean_corpus+=[C]
            CORPUS+=[Lista_Corpus[i]]
    return clean_corpus

In [8]:
def predict_test(modelo,opinion):
    
    texto_as_series = pd.Series(opinion)
    Opinion_Lista = Corpus_lista(texto_as_series)
    Opinion_Vector = Vectoryzer_model.transform(Opinion_Lista)
    Resultado = NB_model.predict(Opinion_Vector)
    
    return Resultado

In [9]:
app = Flask(__name__)

#METODOS
@app.route('/load',methods=['POST'])
def NewData():
    
    datos = request.get_json()
    
    listOfKeys = datos.get("sentences")
    
    list_id =[]
    for x in listOfKeys:
        list_id.append(x.get("id"))
    
    list_content = []
    for x in listOfKeys:
        list_content.append(x.get("content"))
    
    list_classific = []
    for x in listOfKeys:
        list_classific.append(x.get("classification"))
    
    value_as_series = pd.Series(list_classific)
    texto_as_series = pd.Series(list_content)
    New_X = Corpus_lista(texto_as_series)
    
    arreglo_X =[]
    for x in New_X:
        arreglo_X.append(x)
    
    for x in arreglo_X:
        X_balanceado.append(x)
    
    list_classific_as_pd = pd.Series(list_classific)

    New_Y_balanceado = Y_balanceado.append(list_classific_as_pd, ignore_index=True)
    
    clean_array = np.asarray(X_balanceado)
    clean_df = pd.Series(clean_array)
    
    X_fit = Vectoryzer_model.fit(clean_df)
    X_vectorizer = Vectoryzer_model.transform(clean_df)
    
    NB_model.fit(X_vectorizer,New_Y_balanceado)
    
    return 'Entrenamiento realizado'

@app.route('/PredictTest',methods=['POST'])
def Predict():
    datos = request.get_json()
    
    listOfKeys = datos.get("sentences")
    
    list_id =[]
    for x in listOfKeys:
        list_id.append(x.get("id"))
    
    list_content = []
    for x in listOfKeys:
        list_content.append(x.get("content"))
    
    list_classific = []
    for x in listOfKeys:
        list_classific.append(x.get("classification"))
    
    aciertos = 0
    Arreglo_respuesta = []
    prediccion = 0
    TP=0
    TN=0
    FP=0
    FN=0
    i=0
    for x in list_content:
        prediccion = predict_test(NB_model,x)
        if prediccion == 1: 
            if prediccion == list_classific[i]:
                respuesta = {"ID":i+1,
                             "Prediccion":"Este comentario es una queja"
                            }
                Arreglo_respuesta.append(respuesta)
                aciertos += 1
                TP += 1
            if prediccion != list_classific[i]:
                respuesta = {"ID":i+1,
                             "Prediccion":"Este comentario es una queja"}
                Arreglo_respuesta.append(respuesta)
                FP += 1
        else: 
            if prediccion == list_classific[i]:
                respuesta = {"ID:":i+1,
                            "Prediccion":"Es un comentario neutro o positivo"}
                Arreglo_respuesta.append(respuesta)
                aciertos += 1
                TN += 1
            if prediccion != list_classific[i]:     
                respuesta = {"ID: ":i+1,
                             "Prediccion":"Es un comentario neutro o positivo "}
                Arreglo_respuesta.append(respuesta)
                FN += 1
        i += 1
    print(Arreglo_respuesta)
    return jsonify(Arreglo_respuesta)

In [12]:
@app.route('/PredictMetrics',methods=['POST'])
def Metrics():
    datos = request.get_json()
    
    listOfKeys = datos.get("sentences")
    
    list_id =[]
    for x in listOfKeys:
        list_id.append(x.get("id"))
    
    list_content = []
    for x in listOfKeys:
        list_content.append(x.get("content"))
    
    list_classific = []
    for x in listOfKeys:
        list_classific.append(x.get("classification"))
    
    aciertos = 0
    Arreglo_respuesta = []
    prediccion = 0
    TP=0
    TN=0
    FP=0
    FN=0
    i=0
    for x in list_content:
        prediccion = predict_test(NB_model,x)
        if prediccion == 1: 
            if prediccion == list_classific[i]:
                respuesta = {"1.- ID":i+1, 
                             "2.- Clasificacion":str(list_classific[i])+" (Queja)",
                             "3.- Prediccion":"Este comentario es una queja",
                             "4.- Tipo":"True Positive"}
                Arreglo_respuesta.append(respuesta)
                TP += 1
            if prediccion != list_classific[i]:
                respuesta = {"1.- ID":i+1,
                             "2.- Clasificacion":str(list_classific[i])+" (No Queja)",
                             "3.- Prediccion":"Este comentario es una queja",
                             "4.- Tipo":"False Positive"}
                Arreglo_respuesta.append(respuesta)
                FP += 1
        else: 
            if prediccion == list_classific[i]:
                respuesta = {"1.- ID:":i+1,
                             "2.- Clasificacion":str(list_classific[i])+" (No Queja)",
                             "3.- Prediccion":"Es un comentario neutro o positivo",
                             "4.- Tipo":"True Negative"}
                Arreglo_respuesta.append(respuesta)
                aciertos += 1
                TN += 1
            if prediccion != list_classific[i]:     
                respuesta = {"1.- ID: ":i+1,
                             "2.- Clasificacion":str(list_classific[i])+" (Queja)",
                             "3.- Prediccion":"Es un comentario neutro o positivo ",
                             "4.- Tipo":"False Negative"}
                Arreglo_respuesta.append(respuesta)
                FN += 1
        i += 1
    metricas = {
    "Sensibilidad" : TP/(TP+FN),
    "Especificidad" : TN/(TN+FP),
    "Precision" : TP/(TP+FP),
    "Accuracy" : (TP+TN)/(TP+TN+FP+FN),
    "Balanced_Accuracy" : (TP/(TP+FN) + TN/(TN+FP))/2}
    Arreglo_respuesta.append(metricas)
    return jsonify(Arreglo_respuesta)

In [13]:
if __name__ == '__main__':
    
    app.run(debug=True, use_reloader=False, port=8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2021 16:53:13] "POST //PredictMetrics HTTP/1.1" 200 -
